<h1>Questions to be handed in on integration:</h1>

<p>To get started, we load the <code>Gadfly</code> backend for <code>Plots</code>, so that we can make plots; and load the <code>Roots</code> package for <code>D</code>; and the <code>SymPy</code> package:</p>

In [ ]:
using Plots
gadfly()
using Roots			# for D and fzero
using SymPy

<h3>Quick background</h3>

<p>Read about this material here: <a href="http://mth229.github.io/integration.html">integration</a>.</p>

<p>For the impatient, in many cases, the task of evaluating a definite integral is made easy by the fundamental theorem of calculus which says that for a continuous function &#36;f&#36; the following holds for any antiderivate, &#36;F&#36;, of &#36;f&#36;:</p>

&#36;~
\int_a^b f&#40;x&#41; dx &#61; F&#40;b&#41; - F&#40;a&#41;.
~&#36;

<p>That is the definite integral is found by evaluating a related function at the endpoints, &#36;a&#36; and &#36;b&#36;.</p>

<p>The <code>SymPy</code> package can compute many antiderivatives using a version of the <a href="http://en.wikipedia.org/wiki/Risch\_algorithm">Risch algorithm</a> that works for <a href="http://en.wikipedia.org/wiki/Elementary\_function">elementary functions</a>. <code>SymPy</code>'s <code>integrate</code> function can be used to find an indefinite integral:</p>

In [ ]:
f(x) = x^2
integrate(f)

<p>Or a definite integral:</p>

In [ ]:
integrate(f, 0, 1)		# returns a "symbolic" number

<p>However, this only works <em>if</em> there is a known antiderivative &#36;F&#40;x&#41;&#36; – which is not always the case. If not, what to do?</p>

<p>In this case, we can appeal to the definition of the definite integral. For continuous, non-negative &#36;f&#40;x&#41;&#36;, the definite integral is the area under the graph of &#36;f&#36; over the interval &#36;&#91;a,b&#93;&#36;. For possibly negative functions, the indefinite integral is found by the _signed_ area under &#36;f&#36;.  This area can be directly <em>approximated</em> using Riemann sums, or some other approximation scheme.</p>

<p>The Riemann approximation can help. The following pattern will compute a Riemann sum using right-hand endpoints:</p>

In [ ]:
f(x) = x^2
a, b, n = 0, 1, 5		# 5 partitions of [0,1] requested
delta = (b - a)/n		# size of partition
xs = a + (1:n) * delta	
fxs = [f(x) for x in xs]
sum(fxs * delta)		# a new function `sum` to add up values in a container

0.44000000000000006

<p>That value isn't very close to &#36;1/3&#36;. But we only took &#36;n&#61;5&#36; rectangles – clearly there will be some error. Bigger &#36;n&#36;s mean better approximations:</p>

In [ ]:
f(x) = x^2
a, b, n = 0, 1, 50_000		# 50,000 partitions of [0,1] requested
delta = (b - a)/n		
xs = a + (1:n) * delta	
fxs = [f(x) for x in xs]
sum(fxs * delta)

0.3333433334

<p>Note that only the first two lines needed changing to adjust to a new problem. As the pattern is similar, it is fairly easy to wrap the computations in a function for convenience. We borrow this more elaborate one from the notes that works for some other methods beside the default right-Riemann sum:</p>

In [ ]:
function riemann(f::Function, a::Real, b::Real, n::Int; method="right")
  if method == "right"
     meth(f,l,r) = f(r) * (r-l)
  elseif method == "left"
     meth(f,l,r) = f(l) * (r-l)
  elseif method == "trapezoid"
     meth(f,l,r) = (1/2) * (f(l) + f(r)) * (r-l)
  elseif method == "simpsons"
     meth(f,l,r) = (1/6) * (f(l) + 4*(f((l+r)/2)) + f(r)) * (r-l)
  end

  xs = a + (0:n) * (b-a)/n
  as = [meth(f, l, r) for (l,r) in zip(xs[1:end-1], xs[2:end])]
  sum(as)
end

riemann (generic function with 1 method)

<p>The Riemann sum is very slow to converge here. There are faster algorithms both mathematically and computationally. We will discuss two: the trapezoid rule, which replaces rectangles with trapezoids; and Simpson's rule which is a quadratic approximation.</p>

In [ ]:
f(x) = x^2
riemann(f, 0, 1, 1000, method="trapezoid"), riemann(f, 0, 1, 1000, method="simpsons")

(0.33333350000000006,0.3333333333333337)

<p>Base <code>julia</code> provides the <code>quadgk</code> function which uses a different approach altogether. It is used quite easily:</p>

In [ ]:
f(x) = x^2
ans, err = quadgk(f, 0, 1)

(0.3333333333333333,5.551115123125783e-17)

<p>The <code>quadgk</code> function returns two values, an answer and an estimated maximum possible error.  The ans is the first number, clearly it is &#36;1/3&#36;, and the estimated maximum error is the second. In this case it is small (&#36;10^&#123;-17&#125;&#36;) – basically 0.</p>

<h3>Questions</h3>

<ul>
<li>Let &#36;g&#40;x&#41; &#61; x^4 &#43; 10x^2 - 60x &#43; 71&#36;. Find the integral &#36;\int_0^1   g&#40;x&#41; dx&#36; by hand by finding an antiderivative and then using the fundamental theorem of calculus.</li>
</ul>

<ul>
<li>For &#36;f&#40;x&#41; &#61; x/\sqrt&#123;g&#40;x&#41;&#125;&#36; (for &#36;g&#40;x&#41;&#36; from the last problem) estimate the following using 1000 Riemann sums:</li>
</ul>

&#36;~
\int_0^1 f&#40;x&#41; dx
~&#36;

<ul>
<li>Let &#36;f&#40;x&#41; &#61; \sin&#40;\pi x^2&#41;&#36;. Estimate &#36;\int_0^1 f&#40;x&#41; dx&#36; using 20 right-Riemann sums</li>
</ul>

<ul>
<li>For the same &#36;f&#40;x&#41;&#36;, compare your estimate with 20 Riemann sums to   that with 20,000 Riemann sums. How many digits after the decimal   point do they agree?</li>
</ul>

<h4>Left Riemann</h4>

<p>The left Riemann sum uses left-hand endpoints, not right-hand ones. </p>

<ul>
<li>For &#36;f&#40;x&#41; &#61; e^&#123;x&#125;&#36; use the left Riemann sum with &#36;n&#61;10,000&#36; to estimate &#36;\int_0^1 f&#40;x&#41; dx&#36;.</li>
</ul>

<ul>
<li>The left and right Riemann sums for an increasing function are also   lower and upper bounds for the answer. Find the difference between   the left and right Riemann sum for &#36;\int_0^1 e^x dx&#36; when   &#36;n&#61;10,000&#36;. (Use your last answer.) What is the approximate value   &#36;1/100&#36;, &#36;1/1000&#36;, &#36;1/10000&#36;, or &#36;1/100000&#36;?</li>
</ul>

<h4>Trapezoid, Simpson's</h4>

<ul>
<li>The answer to &#36;\int_0^1 e^x dx&#36; is simply &#36;e^1 - e^0&#36; =   &#36;e-1&#36;. Compare the error (in absolute value) of the trapezoid method when &#36;n&#61;10,000&#36;.</li>
</ul>

<ul>
<li>The answer to &#36;\int_0^1 e^x dx&#36; is simply &#36;e^1 - e^0&#36; =   &#36;e-1&#36;. Compare the error of the Simpson's method when &#36;n&#61;10,000&#36;.</li>
</ul>

<p>(The error for Riemann sums is "like" &#36;1/n&#36;, the error for trapezoid sums is like &#36;1/n^2&#36;, and for Simpson's rule the error is like &#36;1/n^4&#36;.)</p>

<h2>quadgk</h2>

<ul>
<li>Use <code>quadgk</code> to find &#36;\int_&#123;-3&#125;^&#123;3&#125; &#40;1 &#43; x^2&#41;^&#123;-1&#125; dx&#36;. What is the answer? What is the estimated maximum error?</li>
</ul>

<p>The answer is:</p>

<p>The error is about</p>

<ul>
<li>Use <code>quadgk</code> to find the integral of &#36;e^&#123;-|x|&#125;&#36; over &#36;&#91;-1,1&#93;&#36;.</li>
</ul>

<ul>
<li>The integral of &#36;\sqrt&#123;1-x^4&#125;&#36; over &#36;&#91;-1,1&#93;&#36; can not be found with the Fundamental Theorem of Calculus using an elementary function for an antiderivative. What is the <em>approximate</em> value?</li>
</ul>

<ul>
<li>The integral of &#36;f&#40;x&#41; &#61; \log&#40;log&#40;x&#41;&#41;&#36; over &#36;&#91;e,e^2&#93;&#36; can not be   found with the Fundamental Theorem of Calculus using an elementary   function for an antiderivative. What is the <em>approximate</em> value?</li>
</ul>

<p>The graph of &#36;f&#40;x&#41;&#36; over the interval &#36;&#91;e, e^2&#93;&#36; makes clear that the triangle formed by the line connecting &#36;&#40;e, f&#40;e&#41;&#41;&#36; and &#36;&#40;e^2, f&#40;e^2&#41;&#41;&#36;, the &#36;x&#36; axis, and the line &#36;x&#61;f&#40;e^2&#41;&#36; will form a lower bound for the area under &#36;f&#36;. What is the error in this approximation? (error = answer &#36;-&#36; approximation)</p>

<ul>
<li>A formula to compute the length of a the graph of the function &#36;f&#40;x&#41;&#36; from &#36;a&#36; to &#36;b&#36; is given by the formula:</li>
</ul>

&#36;~
\int_a^b \sqrt&#123;1 &#43; f&#39;&#40;x&#41;^2&#125; dx
~&#36;

<p>Use this formula when &#36;f&#40;x&#41; &#61; \sin&#40;x&#41;&#36; and the interval is &#36;&#91;0,\pi&#93;&#36;. What is the answer?</p>

<p>Repeat, when the function is &#36;f&#40;x&#41; &#61; x^x&#36; over &#36;&#40;0, 2&#41;&#36;:</p>

<ul>
<li>Compute the area between the intersection points of the two curves &#36;f&#40;x&#41; &#61; x&#36; and &#36;g&#40;x&#41; &#61; x^4&#36; by taking the difference between two definite integrals.</li>
</ul>

<h2>Applications</h2>

<p>We discuss an application of the integral to finding the volumes – not just areas.</p>

<p>A _solid of revolution_ is a figure with rotational symmetry around some axis, such as a soda can, a snow cone, a red solo cup, and other common figures. A formula for the volume of a figure with rotational symmetry can be written in terms of an integral based on a function, &#36;r&#40;h&#41;&#36;, which specifies the radius for various values of &#36;h&#36;.</p>

<blockquote>
<p>If the radius as a function of height is given by &#36;r&#40;h&#41;&#36;, the the volume is &#36;\int_a^b \pi r&#40;h&#41;^2 dh&#36;.</p>
</blockquote>

<p>So for example, a baseball has a overall diameter of &#36;2\cdot 37&#36;mm, but if we place the center at the origin, its rotational radious is given by &#36;r&#40;h&#41; &#61; &#40;37^2 - h^2&#41;^&#123;1/2&#125;&#36; for &#36;-37 \leq h \leq 37&#36;. The volume in mm&#36;^3&#36; is given by:</p>

In [ ]:
r(h) = (37^2 - h^2)^(1/2)
v(h) = pi * r(h)^2
quadgk(v, -37, 37)

(212174.79024304505,2.9103830456733704e-11)

<h3>Glass half full</h3>

<ul>
<li>A glass is formed as a volume of revolution with radius as a   function of height given the equation &#36;r&#40;h&#41; &#61; 2 &#43; &#40;h/20&#41;^4&#36;. The   volume as a function of height &#36;b&#36; is given by &#36;V&#40;b&#41; &#61; \int_0^b \pi   r&#40;h&#41;^2 dh&#36;. Find &#36;V&#40;25&#41;&#36;. Show your work.</li>
</ul>

<ul>
<li>Find a value of &#36;b&#36; so that &#36;V&#40;b&#41; &#61; 455&#36;.</li>
</ul>

<ul>
<li>Now find a value of &#36;b&#36; for which &#36;V&#40;b&#41; &#61; 455/2&#36;. This height will   have half the volume as the height just found. Compare the two   values. Is the ratio of smallest to largest 1/2, more than 1/2 or   less?</li>
</ul>